# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

In [1]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

In [2]:
import collections

import helper
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.layers import GRU, SimpleRNN, Input, Dense, Dropout, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Verify access to the GPU
The following test applies only if you expect to be using a GPU, e.g., while running in a Udacity Workspace or using an AWS instance with GPU support. Run the next cell, and verify that the device_type is "GPU".
- If the device is not GPU & you are running from a Udacity Workspace, then save your workspace with the icon at the top, then click "enable" at the bottom of the workspace.
- If the device is not GPU & you are running from an AWS instance, then refer to the cloud computing instructions in the classroom to verify your setup steps.

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15927472560500858025
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6701206733
locality {
  bus_id: 1
}
incarnation: 9844851794460300678
physical_device_desc: "device: 0, name: GeForce GTX 1070 Ti, pci bus id: 0000:41:00.0, compute capability: 6.1"
]


## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [4]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [5]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les Ã©tats-unis est gÃ©nÃ©ralement froid en juillet , et il gÃ¨le habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [6]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [7]:
import keras 

def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # Cited: Was a little confused with implementation: https://discussions.udacity.com/t/tokenizer-question/321994/4?u=suprachargers2d2n
    # Cited: How to use Tokenizer class: https://stackoverflow.com/questions/41602258/tokenizer-error-in-keras-pipeline
    # Create Tokenizer Object & Fit Text
    Tokeniz = Tokenizer()
    Tokeniz.fit_on_texts(x)
    # Return Tokenized Text & Tokenizer Object
    return Tokeniz.texts_to_sequences(x), Tokeniz
tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [8]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """ 
    # Post Pad the sequences to a given length or ungiven in which it will use the max sequence
    return pad_sequences(x, maxlen=length, padding='post')
tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [9]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 345


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [10]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [11]:
# Learning Rate
learning_rate = 0.001

In [20]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size, verbose=False):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # Build the layers
    # 1 Layer SimpleRNN Network with Softmax activation
    model = Sequential()
    model.add(SimpleRNN(french_vocab_size, activation='softmax', input_shape=input_shape[1:], return_sequences=True))
    # Compile with sparse_categorical_crossentropy & Adam
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    if verbose: model.summary()
    return model
tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(tmp_x.shape, max_french_sequence_length, english_vocab_size, french_vocab_size, verbose=True)
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print('\nPREDICTIONS(Simple RNN):')
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_2 (SimpleRNN)     (None, 21, 345)           119715    
Total params: 119,715
Trainable params: 119,715
Non-trainable params: 0
_________________________________________________________________
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 28s 251us/step - loss: 6.1269 - acc: 0.0244 - val_loss: nan - val_acc: 0.0344
Epoch 2/10
110288/110288 [==============================] - 26s 237us/step - loss: 5.2902 - acc: 0.1007 - val_loss: nan - val_acc: 0.4605
Epoch 3/10
110288/110288 [==============================] - 26s 239us/step - loss: 5.0174 - acc: 0.4578 - val_loss: nan - val_acc: 0.4502
Epoch 4/10
110288/110288 [==============================] - 26s 238us/step - loss: 4.7939 - acc: 0.4600 - val_loss: nan - val_acc: 0.4570
Epoch 5/10
110288/110288 [==============================] - 2

### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [21]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size, verbose=False):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # GRU 1 Layer network with an Added Embedding initial Layer
    # Cited: https://discussions.udacity.com/t/capstone-project-trouble-handling-input-output-shapes-with-embedding/308894/3
    model = Sequential()
    model.add(Embedding(english_vocab_size, 128, input_shape=input_shape[1:]))
    model.add(GRU(french_vocab_size, activation='softmax', return_sequences=True))
    #model.add(Embedding(128, french_vocab_size, input_length=input_shape[1]))
    # Cited: used from simple_model cell above
    # Compile with sparse_categorical_crossentropy & Adam
    model.compile(loss=sparse_categorical_crossentropy, optimizer=Adam(learning_rate), metrics=['accuracy'])
    if verbose: model.summary()
    return model
tests.test_embed_model(embed_model)


# Reshape the input: from 3D Array to 2D
tmp_x_2d = np.reshape(tmp_x, (-1, tmp_x.shape[1]))
# Train the neural network
# Cited: used from simple_model cell above
embed_rnn = embed_model(tmp_x_2d.shape, max_french_sequence_length, english_vocab_size, french_vocab_size, verbose=True)
embed_rnn.fit(tmp_x_2d, preproc_french_sentences, batch_size=512, epochs=10, validation_split=0.2)
# TODO: Print prediction(s)
print('\nPREDICTIONS(Embed RNN):')
print(logits_to_text(embed_rnn.predict(tmp_x_2d[:1])[0], french_tokenizer))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 21, 128)           25472     
_________________________________________________________________
gru_9 (GRU)                  (None, 21, 345)           490590    
Total params: 516,062
Trainable params: 516,062
Non-trainable params: 0
_________________________________________________________________
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 45s 406us/step - loss: 3.2587 - acc: 0.6053 - val_loss: nan - val_acc: 0.6665
Epoch 2/10
110288/110288 [==============================] - 43s 394us/step - loss: 0.9644 - acc: 0.6685 - val_loss: nan - val_acc: 0.6730
Epoch 3/10
110288/110288 [==============================] - 43s 391us/step - loss: 0.8349 - acc: 0.6797 - val_loss: nan - val_acc: 0.6857
Epoch 4/10
110288/110288 [==============================] - 43s 389

### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [22]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size, verbose=False):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # 2 Layer RNN Network with one GRU Bidirectional Layer & the other Dense TimeDistributed Layer
    model = Sequential()
    model.add(Bidirectional(GRU(128, activation='elu', return_sequences=True), input_shape=input_shape[1:]))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))
    # Compile with sparse_categorical_crossentropy & Adam
    # Cited: used from simple_model cell above
    model.compile(loss=sparse_categorical_crossentropy, optimizer=Adam(learning_rate), metrics=['accuracy'])
    if verbose: model.summary()
    return model
tests.test_bd_model(bd_model)


# Train and Print prediction(s)
# Cited: used from simple_model cell above
# Train the neural network
bd_rnn = bd_model(tmp_x.shape, max_french_sequence_length, english_vocab_size, french_vocab_size, verbose=True)
bd_rnn.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)
# Print prediction(s)
# Cited: used from simple_model cell above
print('\nPREDICTIONS(BiRNN):')
print(logits_to_text(bd_rnn.predict(tmp_x[:1])[0], french_tokenizer))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_6 (Bidirection (None, 21, 256)           99840     
_________________________________________________________________
time_distributed_2 (TimeDist (None, 21, 345)           88665     
Total params: 188,505
Trainable params: 188,505
Non-trainable params: 0
_________________________________________________________________
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 25s 225us/step - loss: 2.2009 - acc: 0.5481 - val_loss: nan - val_acc: 0.6138
Epoch 2/10
110288/110288 [==============================] - 22s 195us/step - loss: 1.3210 - acc: 0.6342 - val_loss: nan - val_acc: 0.6498
Epoch 3/10
110288/110288 [==============================] - 22s 196us/step - loss: 1.1963 - acc: 0.6582 - val_loss: nan - val_acc: 0.6631
Epoch 4/10
110288/110288 [==============================] - 21s 194

### Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [13]:
from keras.callbacks import ModelCheckpoint
bestPath = 'models/EncDecModel.hdf5'
ecdEpoch = 10

def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size, 
                 layers:list=None, freezeIdx=-1, verbose=False):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    lyrsNone = layers==None
    if layers==None:
        # 3 Layer GRU Network 1 Bidirectional Layer, & the rest GRU
        # Bottleneck for Encoder/Decoder is 64 units
        layers = []
        layers.append(Embedding(english_vocab_size, 128, input_shape=(output_sequence_length,)))
        layers.append(Bidirectional(GRU(128, activation='elu', return_sequences=True))) 
                                    #input_shape=(output_sequence_length,english_vocab_size)))
        layers.append(GRU(64, activation='elu', return_sequences=True))
        layers.append(GRU(french_vocab_size, activation='softmax', return_sequences=True))
    else:
        for lyr in layers: lyr.trainable = True
        if freezeIdx>0: 
            for i in range(freezeIdx+1): layers[i].trainable = False
    # Keras add Layers
    model = Sequential()
    for i in range(len(layers)): model.add(layers[i])
    if verbose: model.summary()
    # Compile with sparse_categorical_crossentropy & Adam
    # Cited: used from simple_model cell above
    model.compile(loss=sparse_categorical_crossentropy, optimizer=Adam(learning_rate), metrics=['accuracy'])
    # Return
    if lyrsNone: return model, layers
    return model
#tests.test_encdec_model(encdec_model)

# ModelCheckpoint would occasionally throw errors, so this Func. was created to make sure the best model was saved
def Fit(epochs, model, FitF):
        acc = 0.
        for e in range(epochs):
            print("Epochs %d/%d:" % (e+1, epochs))
            a = FitF()
            a = a.history['val_acc'][0]
            if a>acc:
                acc = a
                model.save(bestPath)
        return acc

# Turns the index's into oneHot encoded vectors
def one_hot(s, hotLen):
    assert s.shape[-1]==1
    hot = np.zeros(s.shape[:-1] + (hotLen,), float)
    for ib,b in enumerate(s):
        for it,t in enumerate(b):
            idx = t[0]
            hot[ib][it][idx] = 1.
    return hot

# Reshape the input: from 3D Array to 2D
tmp_x_2d = np.reshape(tmp_x, (-1, tmp_x.shape[1]))
## one_hot Vectors from tmp_x as Input in Net
#x_hot = one_hot(tmp_x, english_vocab_size+1)
# Create Model & Checkpointer
#    - english_vocab_size+1: tmp_x contains index's of max 199 which would be out of range with english_vocab_size=199
encdec_rnn, layers = encdec_model(tmp_x_2d.shape, max_french_sequence_length, english_vocab_size, french_vocab_size, verbose=True)
checkpointer = ModelCheckpoint(filepath=bestPath, save_best_only=True)
# Fit itself: same input & output Sequence
Fit(int(ecdEpoch*1.5), encdec_rnn, 
    lambda: encdec_rnn.fit(tmp_x_2d, tmp_x, batch_size=1024, epochs=1, validation_split=0.2, shuffle=True))
encdec_rnn.load_weights(bestPath)
encdec_rnn = encdec_model(tmp_x_2d.shape, max_french_sequence_length, english_vocab_size, french_vocab_size, 
                                  layers, freezeIdx=2)
# Fit Half: Half of the Targets are changed from the same English to French
halfLen = int(len(tmp_x)*0.50)
half = np.append(tmp_x[halfLen:], preproc_french_sentences[:halfLen], axis=0)
Fit(ecdEpoch, encdec_rnn,
    lambda: encdec_rnn.fit(tmp_x_2d, half, batch_size=1024, epochs=1, 
               validation_split=0.2, shuffle=True))
encdec_rnn.load_weights(bestPath)
# Final Fit: All Targets are now all French
for i in range(2):
    if i==1:
        encdec_rnn = encdec_model(tmp_x_2d.shape, max_french_sequence_length, english_vocab_size, french_vocab_size, 
                                  layers, freezeIdx=-1)
    epc = ecdEpoch if i==0 else 40
    Fit(epc, encdec_rnn,
        lambda: encdec_rnn.fit(tmp_x_2d, preproc_french_sentences, batch_size=1024, epochs=1, 
                   validation_split=0.2, shuffle=True))
    encdec_rnn.load_weights(bestPath)

# Print prediction(s)
# Cited: used from simple_model cell above
print('\nPREDICTIONS(EncDec RNN):')
print(logits_to_text(encdec_rnn.predict(tmp_x_2d[:1])[0], french_tokenizer))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 21, 128)           25472     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 21, 256)           197376    
_________________________________________________________________
gru_2 (GRU)                  (None, 21, 64)            61632     
_________________________________________________________________
gru_3 (GRU)                  (None, 21, 345)           424350    
Total params: 708,830
Trainable params: 708,830
Non-trainable params: 0
_________________________________________________________________
Epochs 1/15:
Train on 110288 samples, validate on 27573 samples
Epoch 1/1
110288/110288 [==============================] - 56s 507us/step - loss: 2.8081 - acc: 0.5040 - val_loss: 1.9801 - val_acc: 0.5568
Epochs 2/15:
Train on 110288 samples, validate on 27573 samples


110288/110288 [==============================] - 63s 573us/step - loss: 2.2347 - acc: 0.6057 - val_loss: nan - val_acc: 0.6135
Epochs 3/40:
Train on 110288 samples, validate on 27573 samples
Epoch 1/1
110288/110288 [==============================] - 61s 555us/step - loss: 2.1798 - acc: 0.6208 - val_loss: nan - val_acc: 0.6268
Epochs 4/40:
Train on 110288 samples, validate on 27573 samples
Epoch 1/1
110288/110288 [==============================] - 54s 486us/step - loss: 2.0879 - acc: 0.6344 - val_loss: nan - val_acc: 0.6431
Epochs 5/40:
Train on 110288 samples, validate on 27573 samples
Epoch 1/1
110288/110288 [==============================] - 55s 499us/step - loss: 2.0541 - acc: 0.6393 - val_loss: nan - val_acc: 0.6431
Epochs 6/40:
Train on 110288 samples, validate on 27573 samples
Epoch 1/1
110288/110288 [==============================] - 54s 490us/step - loss: 2.0295 - acc: 0.6443 - val_loss: nan - val_acc: 0.6486
Epochs 7/40:
Train on 110288 samples, validate on 27573 samples
Epoch

### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [23]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size, verbose=False):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # 3 Layer RNN Network: 1st is the Embedding Layer, then Bidirectional GRU, & GRU
    # All GRU's use activation 'elu'
    model = Sequential()
    model.add(Embedding(english_vocab_size, 64, input_shape=(output_sequence_length,)))
    model.add(Bidirectional(GRU(256, activation='linear', return_sequences=True)))
    #model.add(Dropout(0.2))
    #model.add(TimeDistributed(Dense(french_vocab_size, activation='elu')))
    model.add(GRU(french_vocab_size, activation='softmax', return_sequences=True))
    # Compile with sparse_categorical_crossentropy & Adam
    # Cited: used from simple_model cell above
    model.compile(loss=sparse_categorical_crossentropy, optimizer=Adam(learning_rate), metrics=['accuracy'])
    if verbose: model.summary()
    return model
print('Final Model Loaded')

# "AssertionError: Wrong input shape." is Not Correct:
#     1.  The sequence length can not change from 15 to 21
#     2.  The sequence length for englishData & FrenchData is 21, not 15
#     3.  This Bug issue Cited: https://discussions.udacity.com/t/nlp-capstone-projet-help-on-encoder-decoder/398542/6?u=suprachargers2d2n
#     4.  Issue Cited on: https://discussions.udacity.com/t/final-model-output-size-problem/329813/27
#     5.  Embed adds a dimension, so RepeatVector() does not work because then it would be a 4D Tensor
#tests.test_model_final(model_final)

Final Model Loaded


In [24]:
# TODO: Train the final model
# Cited: used from simple_model cell above
final_rnn = model_final(tmp_x_2d.shape, max_french_sequence_length, english_vocab_size, french_vocab_size, verbose=True)
final_rnn.fit(tmp_x_2d, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 21, 64)            12736     
_________________________________________________________________
bidirectional_7 (Bidirection (None, 21, 512)           493056    
_________________________________________________________________
gru_13 (GRU)                 (None, 21, 345)           888030    
Total params: 1,393,822
Trainable params: 1,393,822
Non-trainable params: 0
_________________________________________________________________
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 48s 440us/step - loss: 3.0373 - acc: 0.4422 - val_loss: nan - val_acc: 0.4964
Epoch 2/10
110288/110288 [==============================] - 45s 405us/step - loss: 1.9408 - acc: 0.5449 - val_loss: nan - val_acc: 0.5865
Epoch 3/10
110288/110288 [==============================] - 45s 403us

## Prediction (IMPLEMENTATION)

In [19]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    bestPath='models/final_best.hdf5'
    # Used from above: Reshaping the input to work with a basic RNN & pad with same squences as max (French Sequence)
    # Also, so x (English) & y (French) have the same shape
    x = pad(x, max_french_sequence_length)
    x = x.reshape((-1, y.shape[-2], 1))
    # Reshape from 3D Array, to 2D
    x = np.reshape(x, (-1, x.shape[1]))
    ### Train neural network using model_final
    model = model_final(x.shape, max_french_sequence_length, english_vocab_size, french_vocab_size, verbose=True)
    #hist = model.fit(x, y, batch_size=256, epochs=1, validation_split=0.2, shuffle=True)
    
    # ModelCheckpoint(checkpointer) would occasionally throw errors, & this Func. 
    # was created to make sure the best model was saved
    def Fit(epochs, model, bestPath):
        acc = 0.
        for e in range(epochs):
            print("Epochs %d/%d:" % (e+1, epochs))
            a = model.fit(x, y, batch_size=1024, epochs=1, validation_split=0.2, shuffle=True)
            a = a.history['val_acc'][0]
            if a>acc:     # if greater accuracy, save model
                acc = a
                model.save(bestPath)
        return acc
    
    ### Train neural network using model_final
    bestAcc = Fit(50, model, bestPath)                         # Fit Model
    model.load_weights(bestPath)                               # Load Best Weights
    print('\nLoaded Best Model with Accuracy:', bestAcc)
    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 21, 64)            12736     
_________________________________________________________________
bidirectional_4 (Bidirection (None, 21, 512)           493056    
_________________________________________________________________
gru_7 (GRU)                  (None, 21, 345)           888030    
Total params: 1,393,822
Trainable params: 1,393,822
Non-trainable params: 0
_________________________________________________________________
Epochs 1/50:
Train on 110288 samples, validate on 27573 samples
Epoch 1/1
110288/110288 [==============================] - 47s 428us/step - loss: 3.0829 - acc: 0.4405 - val_loss: nan - val_acc: 0.4803
Epochs 2/50:
Train on 110288 samples, validate on 27573 samples
Epoch 1/1
110288/110288 [==============================] - 45s 407us/step - loss: 1.9819 - acc: 0.5432 - val_loss: nan - val_acc: 0

Epochs 38/50:
Train on 110288 samples, validate on 27573 samples
Epoch 1/1
110288/110288 [==============================] - 44s 401us/step - loss: 0.1345 - acc: 0.9591 - val_loss: nan - val_acc: 0.9598
Epochs 39/50:
Train on 110288 samples, validate on 27573 samples
Epoch 1/1
110288/110288 [==============================] - 44s 401us/step - loss: 0.1194 - acc: 0.9640 - val_loss: nan - val_acc: 0.9623
Epochs 40/50:
Train on 110288 samples, validate on 27573 samples
Epoch 1/1
110288/110288 [==============================] - 45s 407us/step - loss: 0.1190 - acc: 0.9643 - val_loss: nan - val_acc: 0.9620
Epochs 41/50:
Train on 110288 samples, validate on 27573 samples
Epoch 1/1
110288/110288 [==============================] - 44s 403us/step - loss: 0.1117 - acc: 0.9667 - val_loss: nan - val_acc: 0.9630
Epochs 42/50:
Train on 110288 samples, validate on 27573 samples
Epoch 1/1
110288/110288 [==============================] - 45s 405us/step - loss: 0.1172 - acc: 0.9655 - val_loss: nan - val_ac

## Submission
When you're ready to submit, complete the following steps:
1. Review the [rubric](https://review.udacity.com/#!/rubrics/1004/view) to ensure your submission meets all requirements to pass
2. Generate an HTML version of this notebook

  - Run the next cell to attempt automatic generation (this is the recommended method in Workspaces)
  - Navigate to **FILE -> Download as -> HTML (.html)**
  - Manually generate a copy using `nbconvert` from your shell terminal
```
$ pip install nbconvert
$ python -m nbconvert machine_translation.ipynb
```
  
3. Submit the project

  - If you are in a Workspace, simply click the "Submit Project" button (bottom towards the right)
  
  - Otherwise, add the following files into a zip archive and submit them 
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.

In [ ]:
!!python -m nbconvert *.ipynb

## Optional Enhancements

This project focuses on learning various network architectures for machine translation, but we don't evaluate the models according to best practices by splitting the data into separate test & training sets -- so the model accuracy is overstated. Use the [`sklearn.model_selection.train_test_split()`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to create separate training & test datasets, then retrain each of the models using only the training set and evaluate the prediction accuracy using the hold out test set. Does the "best" model change?